https://www.kaggle.com/code/kareemellithy/imdb-50k-movies-reviews-lstm/notebook

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

## Carreguem el dataset en un dataframe

In [2]:
df = pd.read_csv("IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df.shape

(50000, 2)

In [5]:
df['sentiment'].value_counts().reset_index()

,index,sentiment
0,positive,25000
1,negative,25000


## Netejem el dataframe

Eliminem els caràcters innecessaris, etiquetes html, símbols, etc

In [6]:
def remove_numbers(text):
    return re.sub('[0-9]+', '', text)
def remove_html_tags(text):
    return re.sub(r'<[^>]+>', '', text)
def remove_symbols(text):
    return re.sub("[!@#$%^&*(){}£\/'']",'',text)

df['review']= df['review'].apply(lambda x: remove_numbers(x))
df['review']= df['review'].apply(lambda x: remove_html_tags(x))
df['review']= df['review'].apply(lambda x: remove_symbols(x))

In [7]:
print(f"{df['review'][9000]} -> {df['sentiment'][9000]}")

This movie tries to hard to be something that its not....a good movie. It wants you to be fooled from begining to end,But fails.From when it starts to get interesting it falls apart and youre just hoping the ending gives you some clue of just what is going on but it didnt. -> negative


In [8]:
txt = df['review'][9862]
txt


'I enjoyed this movie. Havent seen Andy Griffith in ages and felt he fit this role perfectly. Ive associated him with comedy but am pleased to see that hes versatile.I wasnt troubled that Dottys "anxiety disorder" may not have been verbatim from a psychiatric textbook. There are zillions of whatever-phobias and neuroses, and these can take on a broad variety of quantitative and qualitative forms. She is clearly a sensitive with extra-sensory powers as was understood by the local Indians but not by any Anglos. It is not surprising that this character is vulnerable and nominally eccentric.Although this is taken to be a light "family movie", it is actually more sophisticated than it seems. Also, Hirams twist at the end came as a pleasant surprise to me and tied all the preceding action together in a bundle. Its fun to contemplate the possibility of such spiritual guidance.'

In [9]:
for token in txt:
    print(token)

I
 
e
n
j
o
y
e
d
 
t
h
i
s
 
m
o
v
i
e
.
 
H
a
v
e
n
t
 
s
e
e
n
 
A
n
d
y
 
G
r
i
f
f
i
t
h
 
i
n
 
a
g
e
s
 
a
n
d
 
f
e
l
t
 
h
e
 
f
i
t
 
t
h
i
s
 
r
o
l
e
 
p
e
r
f
e
c
t
l
y
.
 
I
v
e
 
a
s
s
o
c
i
a
t
e
d
 
h
i
m
 
w
i
t
h
 
c
o
m
e
d
y
 
b
u
t
 
a
m
 
p
l
e
a
s
e
d
 
t
o
 
s
e
e
 
t
h
a
t
 
h
e
s
 
v
e
r
s
a
t
i
l
e
.
I
 
w
a
s
n
t
 
t
r
o
u
b
l
e
d
 
t
h
a
t
 
D
o
t
t
y
s
 
"
a
n
x
i
e
t
y
 
d
i
s
o
r
d
e
r
"
 
m
a
y
 
n
o
t
 
h
a
v
e
 
b
e
e
n
 
v
e
r
b
a
t
i
m
 
f
r
o
m
 
a
 
p
s
y
c
h
i
a
t
r
i
c
 
t
e
x
t
b
o
o
k
.
 
T
h
e
r
e
 
a
r
e
 
z
i
l
l
i
o
n
s
 
o
f
 
w
h
a
t
e
v
e
r
-
p
h
o
b
i
a
s
 
a
n
d
 
n
e
u
r
o
s
e
s
,
 
a
n
d
 
t
h
e
s
e
 
c
a
n
 
t
a
k
e
 
o
n
 
a
 
b
r
o
a
d
 
v
a
r
i
e
t
y
 
o
f
 
q
u
a
n
t
i
t
a
t
i
v
e
 
a
n
d
 
q
u
a
l
i
t
a
t
i
v
e
 
f
o
r
m
s
.
 
S
h
e
 
i
s
 
c
l
e
a
r
l
y
 
a
 
s
e
n
s
i
t
i
v
e
 
w
i
t
h
 
e
x
t
r
a
-
s
e
n
s
o
r
y
 
p
o
w
e
r
s
 
a
s
 
w
a
s
 
u
n
d
e
r
s
t
o
o
d
 
b
y
 
t
h
e
 
l
o
c
a
l
 
I
n
d
i
a
n
s
 
b
u
t
 
n
o
t
 
b


In [10]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
df['sentiment']=label.fit_transform(df['sentiment'])
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically theres a family where a little boy J...,0
4,"Petter Matteis ""Love in the Time of Money"" is ...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,Im going to have to disagree with the previous...,0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],test_size=0.25, random_state=42, stratify=df['sentiment'])

In [12]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (37500,)
Shape of X_test:  (12500,)


In [13]:
v = TfidfVectorizer()

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

In [14]:
RFC_model = RandomForestClassifier()

RFC_model.fit(X_train_cv, y_train)

RandomForestClassifier()

In [15]:
y_pred = RFC_model.predict(X_test_cv)

In [16]:
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.84608
              precision    recall  f1-score   support

           0       0.84      0.85      0.85      6250
           1       0.85      0.84      0.85      6250

    accuracy                           0.85     12500
   macro avg       0.85      0.85      0.85     12500
weighted avg       0.85      0.85      0.85     12500

